In [7]:
import sys
import os
import dlib
import glob
import cv2
import argparse
from imutils import face_utils

# Facial landmark 
เป็นการหาตำแหน่ง ของ ตา คิ้ว จมูก ปาก และโครงหน้า และพล็อตจุดในตามโครงสร้าง
ขั้นแรก จะหาตำแหน่งของใบหน้า (Face Detection)และหาขอบของใบหน้า จากนั้นเราก็จะทำการพล็อตจุด ตามโครงสร้างที่ได้รับมา ลงบนใบหน้าในรูป
การวางตำแหน่งของ ["Facial landmark"](https://www.pyimagesearch.com/wp-content/uploads/2017/04/facial_landmarks_68markup.jpg) จากตำอย่างเราใช้การพล็อตจุดแบบ 68 (x, y)-coordinates โดยแต่ล่ะจุดจะถูกกำหนดตำแหน่งเป็น x,y -coordinates
ซึ่งเราสามารถที่จะ พล็อตจุดทั่วทั้งใบหน้า (shape ) หรือจะเลือกการพล็อตในบริเวณที่สนใจ โดยใช้ face_utils.FACIAL_LANDMARKS_IDXS[" "] เลือตำแหน่งที่สนใจ และสุดท้าย ให้สร้างตัวแปร มาเก็บจุด (x, y) ทั้งหมด ของตำแหน่งนั้น  



In [9]:
predictor_path = "shape_predictor_68_face_landmarks.dat"

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

img = cv2.imread("Monja-Radostics-8076_lowRes.jpg",1)
#the locasion for the eyebrows
(blStart, blEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eyebrow"] 
(brStart, brEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eyebrow"]
#the locasion for the jawline
(jStart, jEnd) = face_utils.FACIAL_LANDMARKS_IDXS["jaw"]



dets = detector(img, 1)
print("Number of faces detected: {}".format(len(dets)))
for k, d in enumerate(dets):
    print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
        k, d.left(), d.top(), d.right(), d.bottom()))
    # Get the landmarks/parts for the face in box d.
    
    shape = predictor(img, d)
    shape = face_utils.shape_to_np(shape)  #convert the facial landmark (x, y)-coordinates to a NumPy array
    left_eyebrow = shape[blStart:blEnd]
    right_eyebrow = shape[brStart:brEnd]
    jawline = shape[jStart:jEnd]
    
# Draw the face Detection on the screen.
    x =  d.left()
    y = d.top()
    w = d.right() - d.left()
    h = d.bottom() -d.top()
    cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
    
#loop over the (x, y)-coordinates for the facial landmarks and draw them on the image
#All 68 Point
    for (x, y) in shape: 
        cv2.circle(img, (x, y ), 2, (0, 0, 255), -1)
        
    #plotting from x1,y1 to x17,17    
    for (x, y) in jawline: 
        cv2.circle(img, (x, y ), 3, (255, 0, 0), -1)
    # eyebrow plotting    
    for (x, y) in left_eyebrow: 
        cv2.circle(img, (x, y ), 3, (0, 255, 0), -1)    
    for (x, y) in right_eyebrow: 
        cv2.circle(img, (x, y ), 3, (0, 255, 0), -1) 
    

    
cv2.imshow("Image",img) 
cv2.waitKey(0)
cv2.destroyAllWindows()

Number of faces detected: 1
Detection 0: Left: 138 Top: 188 Right: 361 Bottom: 411


In [1]:
import sys
import os
import dlib
import glob
import cv2
import argparse
from imutils import face_utils

###Playing Video from file
cap=cv2.VideoCapture("3 Year Old Headbanging-a5BuwkqtIa0.webm")


predictor_path = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)
(mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

while True:
    ret, frame= cap.read()
    if ret:
       
        img= cv2.resize( frame, (640,480))
        dets = detector(img, 1)
        #print("Number of faces detected: {}".format(len(dets)))
        for k, d in enumerate(dets):
           
                # Get the landmarks/parts for the face in box d.
            shape = predictor(img, d)
            shape = face_utils.shape_to_np(shape)  #convert the facial landmark (x, y)-coordinates to a NumPy array
            mouth = shape[mStart:mEnd]
            x = d.left()
            y = d.top()
            w = d.right() - d.left()
            h = d.bottom() -d.top()
            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)

            # Draw the face landmarks on the screen.
            #loop over the (x, y)-coordinates for the facial landmarks and draw them on the image (Mouth)

            for (x, y) in mouth:  
                cv2.circle(img, (x, y ), 2, (0, 0, 255), -1)


            # Display the resulting frame
    cv2.imshow('frame',img)

    ch=cv2.waitKey(1)       #ปรับความเร็ว
    if ch & 0xFF== ord('q'):
             break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()


# Eye blink detection
เป็นวิธีการตรวจสอบและการนับจำนวนการกระพริบ โดยใช้ Facial landmark เข้ามาช่วย โดยใช้หลัการง่ายๆคื่อ เมื่อกระพริบตา ระยะห่างของขอบตาในแนวตั้ง จะมีค่าเข้าใกล้ 0 แต่เพื่อลดการแปรในการวัดระยะที่อาจจะเกิดจากการหมุน เราจึงใช้ [Eye aspect ratio](https://vision.fe.uni-lj.si/cvww2016/proceedings/papers/05.pdf) (EAR) ซึ่งใช้อัตราส่วน ระยะห่างระหว่างขอบตาในแนวนอน และระยะห่างหว่างขอบตาในแนวตั้ง มาใช่ในการตรวจสอบ 
เมื่อเราได้ function ที่จะใช้ในการหา EAR แล้ว ขั้นตอนต่อไปคือ การหาใบหน้าภายในภาพ Face Detection 
ขั้นตอนที่สาม เราจะระบุตำแหน่งของตาทั้งสองข้างโดย ใช่  facial landmark (x, y)-coordinates  ในกรณีนี้เราสนใจแค่ตำแหน่งของตา เราจึงทำการพล็อตจุด ณ แค่ตำแหน่งตา

จากนั้นก็มาเริ่มนับ frame ที่มี EAR น้อยกว่า 0.3 และเมื่อใดก็ตามที่ EAR มากกว่า 0.3 แล้ว counter ของ frame มากกว่าหรือเท่ากับ 3 แต่ต้องน้อยกว่า 10 จะถือว่าเป็นการกระพริบตาหนึ่งครั้ง ถ้านอกหนือจากเงื่อนไขนี้ ไม่นับ เพราะถือว่า เป็นการหลับตา หรือการกระพริบการถี่ๆ 

In [11]:
from scipy.spatial import distance as dist
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from imutils import face_utils
import numpy as np
import argparse
import imutils
import time
import dlib
import cv2
import matplotlib.pyplot as plt

EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 3
COUNTER = 0
TOTAL = 0

def eye_aspect_ratio(eye):
    # compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])

    # compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])

    # compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)
    return ear

predictor_path = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

# the locasion for the eyes
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

vs = FileVideoStream("S_8520928440646.mp4").start() #import Viedo
fileStream = True
time.sleep(1)

while True:
    if fileStream and not vs.more():
        break
    
    frame = vs.read()
    frame = imutils.resize(frame, width=450)

    # detect faces in the grayscale frame
    rects = detector(frame, 0)
    
    for k, d in enumerate(rects):

            # Get the landmarks/parts for the face in box d.
        shape = predictor(frame, d)
        shape = face_utils.shape_to_np(shape)
        leftEye = shape[lStart:lEnd]    
        rightEye = shape[rStart:rEnd]     
        
        leftEAR = eye_aspect_ratio(leftEye)         #(x, y)-coordinates for the facial landmarks 
        rightEAR = eye_aspect_ratio(rightEye)
 
        # average the eye aspect ratio together for both eyes
        ear = (leftEAR + rightEAR) / 2.0
        
        
        
        
        x =  d.left()
        y = d.top()
        w = d.right() - d.left()
        h = d.bottom() -d.top()
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
            
        for (x, y) in leftEye: 
            cv2.circle(frame, (x, y ), 2, (0, 0, 255), -1)
                
        for (x, y) in  rightEye: 
            cv2.circle(frame, (x, y ), 2, (0, 0, 255), -1)
            
        # check to see if the eye aspect ratio is below the blink
        # threshold, and if so, increment the blink frame counter
        if ear < EYE_AR_THRESH:
            COUNTER += 1
 
        # otherwise, the eye aspect ratio is not below the blink
        # threshold
        else:
             # if the eyes were closed for a sufficient number of
            # then increment the total number of blinks
            if COUNTER >= EYE_AR_CONSEC_FRAMES and COUNTER < 10 :
                TOTAL += 1

                # reset the eye frame counter
            COUNTER = 0
        # Text output    
        cv2.putText(frame, "Blinks: {}".format(TOTAL), (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        cv2.putText(frame, "Counter: {}".format(COUNTER), (150, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
       
     
    cv2.imshow('frame',frame)
    
    ch=cv2.waitKey(10)       #ปรับความเร็ว
    if ch & 0xFF== ord('q'):
        break
# When everything done, release the capture

vs.stop()
cv2.destroyAllWindows()



# Mouth Detection
เป็นการตรวจสอบการอ้าปาก หุบปาก หรือการยิงฟันโดยใช้หลัการเดียวกับ Eye blink detection คือการใช้ aspect ratio ของจุดขอบล่างของ ริมฝีปากบนและล่าง มาตรวจสอบ โดยตั้งเงื่อนไขว่าถ้าหาก aspect ratio มีค่า น้อยกว่าหรือเท่ากับ 0.05 จะถือว่า เป็นการหุปปาก แต่เมื่อ aspect ratio มากกว่า 0.05 แต่น้อยกว่าหรือเท่ากับ 0.35 แสดงว่าคนในวีดีโอ กำลังยิ้มหรือยิงฟัน และเมื่อใดที่ ที่ aspect ratio มากกว่า 0.35 จะถือเป็นการอ้าปาก และจะทำการวัดระยะความกว้างของการอ้าปากออกมา

In [5]:
from scipy.spatial import distance as dist
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from imutils import face_utils
import numpy as np
import argparse
import imutils
import time
import dlib
import cv2
import matplotlib.pyplot as plt

MOUTH_IS_CLOSE = 0.05
SMILE = 0.35


predictor_path = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

# the locasion for the Mouth
(mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

vs = FileVideoStream("video_Yr2018M7D19h23m8s2.mp4").start() #import Viedo
fileStream = True
time.sleep(1)

while True:
    if fileStream and not vs.more():
        break
    
    frame = vs.read()
    frame = imutils.resize(frame, width=680)

    # detect faces in the grayscale frame
    rects = detector(frame, 0)
    
    for k, d in enumerate(rects):

            # Get the landmarks/parts for the face in box d.
        shape = predictor(frame, d)
        shape = face_utils.shape_to_np(shape)
        lMouth = shape[mStart:mEnd]   
        
        A = dist.euclidean(lMouth[13], lMouth[19])
        B = dist.euclidean(lMouth[14], lMouth[18])
        C = dist.euclidean(lMouth[15], lMouth[17])
 
        D = dist.euclidean(lMouth[12], lMouth[16])

        # compute the  aspect ratio
        mar = (A + B+C) / (3.0 * D)     
        Distance = (A + B+C) / 3.0
       
        
        
        
        x =  d.left()
        y = d.top()
        w = d.right() - d.left()
        h = d.bottom() -d.top()
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
            
        for (x, y) in lMouth: 
            cv2.circle(frame, (x, y ), 1, (0, 0, 255), -1)
                

            

        if mar <= MOUTH_IS_CLOSE:
            cv2.putText(frame, "Mouth is close ", (d.left() - 10, d.top() - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
 
    # otherwise, the MOUTH aspect ratio is 
        
        else:
            if mar <= SMILE:
                cv2.putText(frame, "show the teeth ", (d.left() - 10, d.top() - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2) 
            
            else:
                cv2.putText(frame, "Mouth is open: {:.2f}".format( Distance), (d.left() - 10, d.top() - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            

#         cv2.putText(frame, "MAR: {:.2f}".format(mar), (300, 30),
#             cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

     
    cv2.imshow('frame',frame)
    
    ch=cv2.waitKey(10)       #ปรับความเร็ว
    if ch & 0xFF== ord('q'):
        break
# When everything done, release the capture
#plt.show() 
cv2.destroyAllWindows()
vs.stop()


# Image Face Aligmen
ใช้ในการจัดของประกอบของหน้า  เมื่อหน้ามีการเอียงหรือการหมุน เพื่อให้การวัดขนาดเป็นไปได้ง่าย และแม่นยำมากขึ้น 

โดยเริ่มจากการ Detect ตาทั้งสองข้าง และหาจุดกึ่งกลางของตาแต่ล่ะข้าง และมุมที่กึ่งกลางของตากระทำต่อกัน ขั้นตอนต่อไปคือระบุอัตราส่วนของหน้า โดยให้ อัตราส่วนของระยะห่างระหว่างตา ใน current image เท่ากับ  อัตราส่วนของระยะห่างระหว่างตา ใน desires image  
จากจุดกึ่งกลางของตาทั้งสองข้างเราก็จะสามารถหาจุดที่อยู่กึ่งกลางระหว่างตาทั้งสองข้างได้  และเราจะใช้จุด จุดนี้ในการคำนวณหา Rotation Matrix เมื่อเราได้ Matrix มาแล้วก็นำไป transformation **cv2.warpAffine()**

โดยวิธีการข้างต้นนี้เราสามารถนำเข้ามาจาก **libaries imutils**  จากนั้นก็เลือใช้คำสั่ง **FaceAligner()** จากตัวอย่างเราได้สร้างตัวแปร fa เพื่อทำการเก็บคำสั่งนี้ และเพื่อง่ายต่อการ align หน้า ต้องทำให้ภาพเป็น binary Image เสียก่อน แล้วจะทำการสร้าง loop เพื่อหมุนตาให้อยู่ในแนวเดียวกับ horizontal line จากคำสั้ง **fa.align(img, gray,i)**

In [3]:
#Image Face Aligment
from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb
from imutils.face_utils import shape_to_np
import argparse
import imutils
import dlib
import cv2

predictor_path = "shape_predictor_68_face_landmarks.dat"

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

fa = FaceAligner(predictor, desiredFaceWidth=256)
img = cv2.imread("Monja-Radostics-8076_lowRes.jpg",1)
img = imutils.resize(img, width=800)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
dets = detector(gray, 1)


for k, d in enumerate(dets):

    shape = predictor(img, d)
    shape = shape_to_np(shape)  #convert the facial landmark (x, y)-coordinates to a NumPy array
    for (x, y) in shape: 
        cv2.circle(img, (x, y ), 2, (0, 0, 255), -1)
    for i in dets:
        (x, y, w, h) = rect_to_bb(i)
        faceAligned = fa.align(img, gray,i)
        
        
cv2.imshow("Aligned", faceAligned)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
#Video Face Aligment
from scipy.spatial import distance as dist
from imutils.video import FileVideoStream
from imutils.video import VideoStream

predictor_path = "shape_predictor_68_face_landmarks.dat"

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)
fa = FaceAligner(predictor, desiredFaceWidth=256)

vs = FileVideoStream("video_Yr2018M7D19h23m8s2.mp4").start() #import Viedo
fileStream = True

#cv2.waitKey(10) 
while True:
    if fileStream and not vs.running():
        break
    
    frame = vs.read()
    frame = imutils.resize(frame, width=680)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    dets = detector(gray, 1)

    for k, d in enumerate(dets):
            
        shape = predictor(frame, d)
        shape = shape_to_np(shape)  #convert the facial landmark (x, y)-coordinates to a NumPy array
            
        for (x, y) in shape: 
            cv2.circle(frame, (x, y ), 2, (0, 0, 255), -1)
        
        for i in dets:
            (x, y, w, h) = rect_to_bb(i)
            Aligned = fa.align(frame, gray,i)
        

            # Display the resulting frame
    cv2.imshow("Aligned", Aligned)

    ch=cv2.waitKey(1)       
    if ch == ord('q'):
        break
            
            
            
            
cv2.destroyAllWindows()
vs.stop()
